In [2]:
from gensim.models import KeyedVectors
model = KeyedVectors.load("model_300_20.bin")

In [6]:
model.wv.similarity("batatas","arroz")

0.64632934

In [33]:
import yaml
with open("class_by_simil.yaml", encoding="utf8") as f:
    dict= yaml.safe_load(f)
print(dict)





[{'tit': 'polarity', 'clas': ['bom', 'mau'], 'testes': [['guerra', 'mau'], ['pizza', 'bom'], ['descançar', 'bom'], ['cansado', 'mau']]}, {'tit': 'class-animais', 'clas': ['ave', 'mamífero', 'peixe'], 'testes': [['melro', 'ave'], ['águia', 'ave'], ['pomba', 'ave'], ['gato', 'mamífero'], ['leão', 'mamífero'], ['macaco', 'mamífero'], ['boi', 'mamífero'], ['tubarão', 'peixe'], ['sardinha', 'peixe'], ['salmão', 'peixe']]}]


In [11]:
array = [
    {'tit': 'polarity', 'clas': ['bom', 'mau'], 'testes': [['guerra', 'mau'], ['pizza', 'bom'], ['descançar', 'bom'], ['cansado', 'mau']]},
     {'tit': 'class-animais', 'clas': ['ave', 'mamífero', 'peixe'], 'testes': [['melro', 'ave'], ['águia', 'ave'], ['pomba', 'ave'], ['gato', 'mamífero'], ['leão', 'mamífero'], ['macaco', 'mamífero'], ['boi', 'mamífero'], ['tubarão', 'peixe'], ['sardinha', 'peixe'], ['salmão', 'peixe']]}]


In [59]:
import pandas as pd

words = ['bom', 'ave', 'mamífero', 'peixe', 'mau']
dataframe = pd.DataFrame(array)

def _remove_non_existing_words(row):
    return all([clas in words for clas in row['clas']])
    # return all([clas in model.wv.key_to_index for clas in classes]) Usar isto

def _similarity(row):
    # return model.wv.similarity(row['word'], row['testes']) Usar isto
    return row['answer']


all_words = dataframe["clas"].explode()
unknown_words = all_words[~all_words.isin(words)]
print(unknown_words)

dataframe[dataframe.apply(_remove_non_existing_words, axis=1)] # Remove rows with words not present in the dictionary
dataframe = dataframe.explode('testes') # Dividir 
dataframe[['word', 'answer']] = pd.DataFrame(dataframe['testes'].tolist(), index=dataframe.index)
dataframe = dataframe.explode('clas')
dataframe['predicted'] = dataframe.apply(_similarity, axis= 1)
dataframe

Series([], Name: clas, dtype: object)


,tit,clas,testes,word,answer,predicted
0,polarity,bom,"[guerra, mau]",guerra,mau,mau
0,polarity,mau,"[guerra, mau]",guerra,mau,mau
0,polarity,bom,"[pizza, bom]",pizza,bom,bom
0,polarity,mau,"[pizza, bom]",pizza,bom,bom
0,polarity,bom,"[descançar, bom]",descançar,bom,bom
0,polarity,mau,"[descançar, bom]",descançar,bom,bom
0,polarity,bom,"[cansado, mau]",cansado,mau,mau
0,polarity,mau,"[cansado, mau]",cansado,mau,mau
1,class-animais,ave,"[melro, ave]",melro,ave,ave
1,class-animais,mamífero,"[melro, ave]",melro,ave,ave


In [59]:
score = 0
oov = []
total = 0

for elem in dict:
    if any([clas not in model.wv.key_to_index for clas in elem["clas"]]):
        continue

    for teste in elem['testes']:
        if teste[0] not in model.wv.key_to_index:
            oov.append(teste[0])
            continue
        total += 1
        result = [model.wv.similarity(teste[0],clas) for clas in elem['clas']]
        index = result.index(max(result))
        print(index,teste[0],elem['clas'][index])
        if elem['clas'][index] == teste[1]:
            score += 1
print(score)
print(oov)
print(round(score/total,4))

1 guerra mau
0 pizza bom
0 descançar bom
1 cansado mau
2 melro peixe
0 águia ave
0 pomba ave
2 gato peixe
2 leão peixe
1 macaco mamífero
2 boi peixe
2 tubarão peixe
2 sardinha peixe
2 salmão peixe
10
[]
0.7143


In [16]:
print(model.wv.similarity("bom","ruído"))
print(model.wv.similarity("mau","ruído"))



0.16179302
0.24127741
